In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import face_recognition
from datetime import datetime
from typing import List,Tuple,Optional 
from pkg_resources import resource_filename

In [4]:
# STEP1: Configuration / constants

from smart_attendance_system import(
    KNOWN_FACES_DIR,
    ATTENDANCE_CSV,
    CAMERA_INDEX,
    FACE_MATCH_THRESHOLD,
    SCALE_FACTOR,
    load_and_encode_faces,
    recognize_faces_from_video
)

if __name__ == "__main__":
    if not os.path.exists(KNOWN_FACES_DIR):
        raise SystemExit(f"known face directory not found: {KNOWN_FACES_DIR}. creat it and add subfolders per person.")
    
    encodings, names = load_and_encode_faces(KNOWN_FACES_DIR)
    recognize_faces_from_video(encodings, names, attendance_file=ATTENDANCE_CSV)


#STEP2: Real-time recognition (Camera)

def recognize_faces_from_video(known_encodings: List[np.array],
                               known_names: List[str],
                               attendance_file: str = ATTENDANCE_CSV,
                               camera_index: int = CAMERA_INDEX):
    video_capture = cv2.VideoCapture(camera_index)
    if not video_capture.isOpened():
        print("[ERROR] could not open video source.")
        return
    
    print("[INFO] starting video strem. press q to quit.")

    video_capture = cv2.VideoCapture()

    while True:
            ret, frame = video_capture.read()
            if not ret:
                    break

# Resize for speed
    small_frame = cv2.resize(frame, (0, 0), fx=SCALE_FACTOR, fy=SCALE_FACTOR)

# Convert to RGB and ensure contiguity
    rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    rgb_small = np.ascontiguousarray(rgb_small)

#Detect face and compute encodings
    face_locations = face_recognition.face_locations(rgb_small)
    face_encodings = face_recognition.face_encodings(rgb_small, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
# Compare to known encodings
        if len(known_encodings) > 0:
            distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_idx = np.argmin(distances)
            best_distance = float(distances[best_idx])
            if best_distance <= FACE_MATCH_THRESHOLD:
                    name = known_names[best_idx]
            else:
                name = 'Unknown'
                best_distance = None

# Scale coordinates back to original frame size
            top = int(top / SCALE_FACTOR)
            right = int(right / SCALE_FACTOR)
            bottom = int(bottom / SCALE_FACTOR)
            left = int(left / SCALE_FACTOR)

# Draw box and label
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 0, 0), 1)

# Mark attendance for recognized faces
            if name != 'Unknown':
# Append to CSV (skips duplicates for the same day by default)
                mark_attendance(name, attendance_file)

                cv2.imshow('Smart Attendance', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        video_capture.release()
        cv2.destroyAllWindows()    

[DEBUG] Starting face encoding load process...
[DEBUG] Checking if directory exists: known_faces
[DEBUG] Directory found. Scanning subfolders for people...

[INFO] Processing person: Aji
[DEBUG] Looking into folder: known_faces\Aji
[DEBUG] Loading image: known_faces\Aji\aji.png
[DEBUG] Image loaded successfully. Shape: (485, 412, 3)
[DEBUG] Face encoding computed. Encoding vector length: 128
[INFO] Added encoding for Aji from file aji.png

[INFO] Processing person: Danush
[DEBUG] Looking into folder: known_faces\Danush
[DEBUG] Loading image: known_faces\Danush\danush.png
[DEBUG] Image loaded successfully. Shape: (521, 305, 3)
[DEBUG] Face encoding computed. Encoding vector length: 128
[INFO] Added encoding for Danush from file danush.png

[INFO] Processing person: Jayaseelan
[DEBUG] Looking into folder: known_faces\Jayaseelan
[DEBUG] Loading image: known_faces\Jayaseelan\jayaseelan.png
[DEBUG] Image loaded successfully. Shape: (362, 283, 3)
[DEBUG] Face encoding computed. Encoding vect